In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization, ReLU, Dropout
from tensorflow.keras.models import Model, clone_model
from itertools import product

import functions as fs
from functions import make_state, run, get_optimal_value, get_optimal_action, get_optimal_actions, get_model_actions, test_model, one_batch, test_model_accuracy, one_batch_supervised, train, train_supervised, create_model


# 单步奖励收集

这是一个用于练习策略梯度法 (policy gradient method) 的玩具题。  
8个格子首尾相连，其中随机3个格子奖励，奖励数值为0~1的均匀随机数。  
动作：选择一个格子。
计分规则：  
1. 如果格子有奖励（大于0），获得对应的奖励。
2. 如果格子是空的（等于0），且两侧的格子都有奖励，会获得两侧格子的总奖励。
3. 否则奖励为0。

## 试验场

In [3]:
model = create_model(n_hidden_layers=1, n_dense_units=512, ratio_dropout=0.5, optimizer='adam')

In [4]:
best_idx, best_score, best_weights = train(model, 200, verbose = 1)

0 0.4196964480362878
1 0.533370590878932
2 0.5609241545276226
3 0.5788089194138847
4 0.6162808826571229
5 0.6452382955228987
6 0.655994152861143
7 0.6722184017389673
8 0.6987733853780318
9 0.7171190232594449
10 0.7223539906471552
11 0.7033830109012923
12 0.6959905332599443
13 0.7044713307648616
14 0.7143880538038173
15 0.7154319676800828
16 0.7097366429164729
17 0.7067603394464773
18 0.7058744872277171
19 0.7443789022555416
20 0.7775803586761424
21 0.7762757632392603
22 0.7747033627363588
23 0.7781361266069076
24 0.783626005851184
25 0.788986649470874
26 0.7768568303450715
27 0.7676763491307047
28 0.7875435922075503
29 0.7685812257518093
30 0.7461966768516559
31 0.7541715711196213
32 0.7439933785816666
33 0.7635545243688332
34 0.7953988900566931
35 0.7802354797650374
36 0.7768875566808764
37 0.7761849346229717
38 0.8079974398336426
39 0.8120313718200692
40 0.8113655419591916
41 0.8078333821925159
42 0.8117695504163437
43 0.8177050332178093
44 0.808231075415247
45 0.7989502032726911
46 

In [5]:
best_idx, best_score

(np.int64(88), np.float64(0.892805906106112))

In [6]:
model.set_weights(best_weights)

In [7]:
test_model(model, 10000)

np.float64(0.8903345454723381)

In [8]:
test_model_accuracy(model, 10000)

np.float64(0.4959)

In [17]:
model_supervised = create_model(n_hidden_layers=2, n_dense_units=512, ratio_dropout=0.5, optimizer='adam')

In [18]:
train_supervised(model_supervised, 200, verbose = 1)

0 0.5798255174581393
1 0.6715001091366704
2 0.6762108837108425
3 0.7180279808981633
4 0.7237650492489376
5 0.7151350099839339
6 0.7058486956482112
7 0.7009318060557419
8 0.7166195915181409
9 0.7583921386864083
10 0.7796342298048112
11 0.7991952385965791
12 0.7865246071069473
13 0.7863458954037722
14 0.7921531220020768
15 0.8020982985265503
16 0.8050563632421734
17 0.7843381442977807
18 0.7783986838224585
19 0.7662579167319429
20 0.7735607376108803
21 0.7683413114838522
22 0.7772922221070516
23 0.7752406756752738
24 0.7658300852460661
25 0.7626813411891772
26 0.7643539313020212
27 0.745547003197266
28 0.7619439461651285
29 0.7573852882142701
30 0.7729987892947844
31 0.7829905282565662
32 0.7799546308585512
33 0.7783302276314537
34 0.7750431488453194
35 0.7579681585722092
36 0.7908543785272706
37 0.8123053456390713
38 0.8174631549771002
39 0.8199705108491878
40 0.8100872079188145
41 0.8066552138558288
42 0.8113612039691162
43 0.8031792989392078
44 0.802376837407698
45 0.8035674866996085


In [175]:
test_model(model_supervised, 10000)

np.float64(0.9005773898710413)

In [165]:
test_model_accuracy(model_supervised, 10000)

np.float64(0.769)

## 一些测试结论

先做了一些模拟测试（10万次）确定评价模型的基准。如果采用最优解，单局游戏奖励的平均值为0.933，标准差0.347。采用随机策略的平均奖励是0.297。

首先初步对比了一下有无 BatchNormalization 和 Dropout 的情况，差距颇大。无 BatchNormalization 和 Dropout 时表现很差。观察 weights，应该是遇到了梯度消失和梯度爆炸。

之后的测试都用了 BatchNormalization 和 Dropout。

依次是：optimalizer, n_hidden_layers, n_dense_units, ratio_dropout, argmax(scores), max(scores)，最后两列是监督学习的结果
sgd	1	64	0.2	230	0.81957047	199	0.814762256
sgd	1	64	0.3	133	0.823306626	232	0.846447604
sgd	1	64	0.4	215	0.809455314	199	0.842190759
sgd	1	64	0.5	222	0.856119525	285	0.844487367
sgd	1	64	0.6	219	0.865266425	216	0.833977499
sgd	1	64	0.7	257	0.817717295	233	0.810288408
sgd	1	128	0.2	138	0.848355803	225	0.834754306
sgd	1	128	0.3	233	0.843209246	201	0.837043901
sgd	1	128	0.4	203	0.84398221	286	0.842715486
sgd	1	128	0.5	123	0.884888732	265	0.841131572
sgd	1	128	0.6	222	0.832868491	185	0.844697325
sgd	1	128	0.7	137	0.857809365	222	0.83471746
sgd	1	256	0.2	197	0.878311576	162	0.891374889
sgd	1	256	0.3	155	0.879151885	154	0.878041714
sgd	1	256	0.4	241	0.860009212	211	0.884917987
sgd	1	256	0.5	146	0.878039385	88	0.877335078
sgd	1	256	0.6	199	0.874558249	155	0.872676206
sgd	1	256	0.7	184	0.852593144	193	0.867367797
sgd	1	512	0.2	144	0.900807575	155	0.881127728
sgd	1	512	0.3	157	0.903435378	185	0.902059197
sgd	1	512	0.4	156	0.896634056	148	0.891186373
sgd	1	512	0.5	111	0.885416902	167	0.887317218
sgd	1	512	0.6	145	0.881211699	179	0.896044479
sgd	1	512	0.7	195	0.880886037	171	0.878781404
sgd	2	64	0.2	180	0.829470364	213	0.824875833
sgd	2	64	0.3	213	0.813004653	181	0.810666407
sgd	2	64	0.4	221	0.796345613	165	0.8167897
sgd	2	64	0.5	215	0.82388745	238	0.82007955
sgd	2	64	0.6	195	0.810177596	216	0.806053396
sgd	2	64	0.7	262	0.77908087	221	0.781639501
sgd	2	128	0.2	168	0.810202247	202	0.829993644
sgd	2	128	0.3	160	0.849377718	142	0.852961319
sgd	2	128	0.4	184	0.818962699	168	0.797151086
sgd	2	128	0.5	216	0.823738145	153	0.822228704
sgd	2	128	0.6	170	0.830139236	185	0.815216612
sgd	2	128	0.7	225	0.808585633	162	0.831086245
sgd	2	256	0.2	223	0.821159954	128	0.825304977
sgd	2	256	0.3	168	0.833991425	213	0.814544642
sgd	2	256	0.4	201	0.806464947	179	0.810070847
sgd	2	256	0.5	131	0.833720606	172	0.823969548
sgd	2	256	0.6	143	0.833717145	125	0.840917363
sgd	2	256	0.7	168	0.800307924	216	0.81568297
sgd	2	512	0.2	148	0.814656652	163	0.804054857
sgd	2	512	0.3	175	0.815816799	170	0.797978322
sgd	2	512	0.4	204	0.802997249	143	0.787797558
sgd	2	512	0.5	188	0.787115627	133	0.82032757
sgd	2	512	0.6	148	0.831742408	138	0.843009766
sgd	2	512	0.7	177	0.810344541	125	0.825383229
adam	1	64	0.2	237	0.800325885	106	0.851902918
adam	1	64	0.3	110	0.83046924	145	0.841763447
adam	1	64	0.4	137	0.830828595	152	0.87059485
adam	1	64	0.5	257	0.850197804	135	0.832163489
adam	1	64	0.6	246	0.848341135	235	0.826210482
adam	1	64	0.7	297	0.820871104	272	0.840769845
adam	1	128	0.2	73	0.872352655	79	0.840301851
adam	1	128	0.3	120	0.852311373	99	0.852244187
adam	1	128	0.4	147	0.87223051	88	0.860748639
adam	1	128	0.5	158	0.868205929	95	0.870478781
adam	1	128	0.6	140	0.877279833	119	0.893925984
adam	1	128	0.7	169	0.866685221	236	0.85733735
adam	1	256	0.2	83	0.859628576	95	0.866258281
adam	1	256	0.3	83	0.876542719	78	0.868539565
adam	1	256	0.4	93	0.894967784	91	0.889777236
adam	1	256	0.5	83	0.877598209	100	0.885931925
adam	1	256	0.6	123	0.894390356	98	0.893189216
adam	1	256	0.7	152	0.87269044	143	0.890608802
adam	1	512	0.2	62	0.862585047	70	0.886787742
adam	1	512	0.3	83	0.884045646	72	0.878337055
adam	1	512	0.4	97	0.878438115	66	0.880801083
adam	1	512	0.5	94	0.896152579	72	0.896046623
adam	1	512	0.6	91	0.889568521	102	0.905443719
adam	1	512	0.7	97	0.895427823	121	0.906246792
adam	2	64	0.2	164	0.757905701	271	0.733956743
adam	2	64	0.3	110	0.796601826	140	0.718582735
adam	2	64	0.4	88	0.749958685	107	0.806369852
adam	2	64	0.5	121	0.793363862	152	0.800517474
adam	2	64	0.6	118	0.802693139	124	0.782237715
adam	2	64	0.7	165	0.735777228	199	0.782845587
adam	2	128	0.2	291	0.774913456	110	0.722267079
adam	2	128	0.3	91	0.757426543	105	0.790650053
adam	2	128	0.4	96	0.794715882	129	0.799408182
adam	2	128	0.5	104	0.786028039	122	0.787805528
adam	2	128	0.6	108	0.835592486	119	0.799569394
adam	2	128	0.7	109	0.791363958	106	0.812970546
adam	2	256	0.2	181	0.767824652	165	0.776558887
adam	2	256	0.3	193	0.769441759	289	0.773747808
adam	2	256	0.4	99	0.790121204	251	0.754174716
adam	2	256	0.5	106	0.772241874	96	0.822737305
adam	2	256	0.6	143	0.78152916	95	0.784852849
adam	2	256	0.7	126	0.797849688	101	0.762861092
adam	2	512	0.2	241	0.752311116	125	0.774890214
adam	2	512	0.3	149	0.76133756	122	0.774810565
adam	2	512	0.4	220	0.777328298	137	0.762169157
adam	2	512	0.5	224	0.749306438	155	0.769043194
adam	2	512	0.6	237	0.749582459	106	0.769278361
adam	2	512	0.7	98	0.789538087	167	0.725196807


几个结论：
1. 对比强化学习和监督学习，强化学习算法的实现方式是正确的。
2. adam 比 sgd 更快，且更稳定（没有体现在上面的数据里），为什么？sgd模型的参数绝对值会越来越大，adam似乎没这个问题
3. 对于两个隐藏层的模型，sgd 比 adam 表现更好
4. 两个隐藏层的模型比一个隐藏层的模型效果差。为什么？
5. 为什么增大模型规模能提升模型效果和加快学习速度？看到过一个理论，说模型中大部分参数是没用的，只有部分参数起决定性作用，因为这部分参数的初始值很好。大的模型有更多的参数，有更大机会命中较好的初始参数。

观察了两个强化学习和监督学习模型，各自的得分是0.89和0.9，差距很小，最优解的命中率却是0.49和0.77。考虑：
1. 用 baseline 方法重新训练策略模型试试。
2. 训练过程中逐步降温。

之后调整测试 policy gradient method 采用超参数：optimalizer, n_hidden_layers, n_dense_units, ratio_dropout = 'adam', 1, 512, 0.5






## baseline

将 one_batch 中 
```
y_target[action] = run(state, action)
```
改为
```
y_target[action] = run(state, action) - 0.933
```

模型没有完全失效，训练过程中仍有提升，但只达到了0.72 左右。我看不出为什么在奖励上减去一个固定的数字会影响结果。